In [9]:
!pip install requests beautifulsoup4 chromadb PyPDF2 sentence-transformers transformers huggingface_hub  llama-cpp-python jinja2 python-decouple openai==1.56.1 httpx==0.27.2 SpeechRecognition pydub youtube-transcript-api jinja2 llm-templates litellm[proxy] wikipedia llama-index-llms-ollama llama-index

# 1. Setup
Preparacion de directorios y decarga de documentos correspondientes.

In [10]:
!mkdir data
!chmod 777 data

mkdir: cannot create directory ‘data’: File exists


In [11]:
%cd data
!wget https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/refs/heads/main/data/resena.txt
!wget https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/main/data/lost-ruins-of-arnak-rules-es.pdf
!wget https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/refs/heads/main/data/preguntas_y_respuestas.csv
!wget https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/refs/heads/main/data/arnak_resena_audio.m4a
!wget https://czechgames.com/files/rules/lost-ruins-of-arnak-rules-en.pdf
%cd ..
!wget https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/refs/heads/main/.env

/content/data
--2024-12-18 01:54:29--  https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/refs/heads/main/data/resena.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659 (20K) [text/plain]
Saving to: ‘resena.txt.1’

resena.txt.1        100%[===================>]  20.17K  --.-KB/s    in 0.002s  

2024-12-18 01:54:29 (11.8 MB/s) - ‘resena.txt.1’ saved [20659/20659]

--2024-12-18 01:54:29--  https://raw.githubusercontent.com/giulicrenna/TUIA-NLP/main/data/lost-ruins-of-arnak-rules-es.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200

In [12]:
# Librerías de uso general
import os
import json
import requests
import dotenv
import re
import numpy as np
from google.colab import userdata

#Librerías para el preprocesamiento de datos
import csv
import chromadb
import wikipedia
import pandas as pd
import speech_recognition as sr
from youtube_transcript_api import YouTubeTranscriptApi
from pydub import AudioSegment
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup
from chromadb.config import Settings

# Librerías para la creación del modelo con Random Forest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Librerias para la creación del RAG
import torch
from openai import OpenAI
from decouple import config
from transformers import pipeline
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Librerías para el agente
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.formatter import ReActChatFormatter
from llama_index.core.agent import ReActAgent

In [13]:
import warnings
warnings.filterwarnings('ignore')

# 1. Preparación de los datos

## 1.1 Scrapping
Se realiza el scrapping de las páginas donde se encuentran la información requerida del juego.

In [14]:
def save_webpage_text(url, output_file):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        full_text = soup.get_text(separator="\n", strip=True)

        with open(output_file, 'w', encoding='utf-8') as file:
          if not os.path.exists(output_file):
            file.write(full_text)

        print(f"Texto extraído y guardado en {output_file}")
    except Exception as e:
        print(f"Error: {e}")

In [15]:
save_webpage_text(url="https://misutmeeple.com/2021/04/resena-las-ruinas-perdidas-de-arnak/",
                  output_file=os.path.join(os.getcwd(), 'data', 'resena.txt'))

Texto extraído y guardado en /content/data/resena.txt


In [16]:
save_webpage_text(url="https://donmeeple.com/ruinas-perdidas-arnak-juego-mesa/",
                  output_file=os.path.join(os.getcwd(), 'data', 'resena_2.txt'))

Texto extraído y guardado en /content/data/resena_2.txt


In [17]:
save_webpage_text(url="https://mishigeek.com/lost-ruins-of-arnak-resena-en-espanol/",
                  output_file=os.path.join(os.getcwd(), 'data', 'reglas.txt'))

Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


## 1.2 Creación de la base de datos vectorial
Creo todas las funciones necesarias para facilitar el preprocesado de los datos.

In [37]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
client = chromadb.PersistentClient(path=os.path.join('data', 'vector'))

collection = client.get_or_create_collection(name="arnak")

In [20]:
def read_csv_file(file_path: str) -> list[dict]:
    """
    Lee un archivo CSV y devuelve una lista de diccionarios, donde cada diccionario representa una fila.
    """
    rows = []

    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            rows.append(row)

    return rows

In [21]:
def read_text_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [22]:
def read_pdf_file(file_path: str) -> str:
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [23]:
def transcribir(url: str) -> str:
    try:
        video_id_match = re.search(r"v=([a-zA-Z0-9_-]+)", url)
        if video_id_match:
            video_id = video_id_match.group(1)
        else:
            raise ValueError("No se pudo extraer el ID del video de la URL.")

        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=('es', 'en'))
        text = " ".join([entry['text'] for entry in transcript])

        text = text.replace('\n', ' ').strip()

        return text

    except Exception as e:
        print("Error:", e)


In [24]:
def split_audio(file_path: str, chunk_length_ms: int = 30000):
    """
    Divide un archivo de audio en fragmentos más pequeños.
    """
    audio = AudioSegment.from_file(file_path)
    chunks = []

    for i, start in enumerate(range(0, len(audio), chunk_length_ms)):
        chunk = audio[start:start + chunk_length_ms]
        chunk_name = f"chunk_{i}.wav"
        chunk.export(chunk_name, format="wav")
        chunks.append(chunk_name)

    return chunks

In [25]:
def m4a_to_text(chunks: list) -> str:
    """
    Convierte un archivo de audio .m4a a texto.
    """
    recognizer = sr.Recognizer()

    text_result = ""

    for i, chunk_path in enumerate(chunks):
        with sr.AudioFile(chunk_path) as source:
            print(f"Analizando fragmento {i + 1}...")
            audio_data = recognizer.record(source)
            try:
                text_result += recognizer.recognize_google(audio_data, language="es-ES") + " "
            except sr.UnknownValueError:
                text_result += "[Incomprensible] "
            os.remove(chunk_path)

    return text_result

In [26]:
def split_into_chunks(text: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list[str]:
    clean_text = "\n".join(
        line.strip() for line in text.splitlines() if line.strip()
    )

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", "."]
    )

    chunks = text_splitter.split_text(clean_text)

    return chunks

In [27]:
def load_csv_into_to_chromadb(csv_path: str) -> None:
    """
    Lee un archivo CSV, transforma los datos y los indexa en una colección de ChromaDB.
    """
    csv_data = read_csv_file(csv_path)

    # Formatear los datos para ChromaDB (id y texto)
    for idx, row in enumerate(csv_data):
        question = row.get("Pregunta", "")  # Asegúrate de que esta columna exista en tu CSV
        answer = row.get("Respuesta", "")   # Asegúrate de que esta columna exista en tu CSV
        if question and answer:
            collection.add(
                ids=[f"doc_{idx}"],
                documents=[question],
                metadatas=[{"answer": answer}]
            )

In [28]:
def clean_chromadb(collection_name: str = None):
  try:
      client.delete_collection(collection_name)
      print(f"Colleccion '{collection_name}' borrada.")
  except Exception as e:
      print(f"Error eliminando '{collection_name}': {e}")

In [29]:
def insert_into_chromadb(collection: str, chunks: list[str], embedding_model) -> None:
    for i, chunk in enumerate(chunks):
        embedding = embedding_model.encode(chunk)
        collection.add(
            documents=[chunk],
            metadatas=[{"chunk_id": i}],
            ids=[f"doc_{i}"]
        )

In [30]:
def process_and_store(file_paths: str) -> None:
  for file_path in file_paths:
      print(f'Embebiendo datos en la BD Vectorial.\n', '*'*50)
      try:
          print(f'Procesando: {file_path}')
          if file_path.endswith('.txt'):
              text = read_text_file(file_path)
          elif file_path.endswith('.pdf'):
              text = read_pdf_file(file_path)
          elif file_path.endswith('.csv'):
              load_csv_into_to_chromadb(file_path)
              text = ""
          elif 'https://' in file_path:
              text = transcribir(file_path)
          else:
            raise ValueError(f"Formato no soportado: {file_path}")
          chunks = split_into_chunks(text)
          insert_into_chromadb(collection, chunks, embedding_model)
      except Exception as e:
        print(e)
        pass

In [31]:
file_paths: list[str] = [os.path.join(os.getcwd(), 'data', 'resena.txt'),
                         os.path.join(os.getcwd(), 'data', 'preguntas_y_respuestas.csv'),
                         os.path.join(os.getcwd(), 'data', 'lost-ruins-of-arnak-rules-en.pdf'),
                         os.path.join(os.getcwd(), 'data', 'lost-ruins-of-arnak-rules-es.pdf'),
                         'https://www.youtube.com/watch?v=FpKxw-VwZSg',
                         'https://www.youtube.com/watch?v=lY_x3mbLRsY']

In [32]:
process_and_store(file_paths)

Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: /content/data/resena.txt
Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: /content/data/preguntas_y_respuestas.csv


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 31.9MiB/s]


Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: /content/data/lost-ruins-of-arnak-rules-en.pdf


Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: /content/data/lost-ruins-of-arnak-rules-es.pdf


Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: https://www.youtube.com/watch?v=FpKxw-VwZSg


Embebiendo datos en la BD Vectorial.
 **************************************************
Procesando: https://www.youtube.com/watch?v=lY_x3mbLRsY


## 1.2 Funciones de consulta

In [33]:
def get_embedding(query: str, embedding_model) -> list[float]:
    return embedding_model.encode(query)

In [34]:
def query_chromadb(collection: str, query: str, embedding_model, top_k: int=3) -> list[list[float]]:
    embedding = get_embedding(query, embedding_model)
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k
    )
    return results

In [35]:
def display_results(results):
    for i, document in enumerate(results['documents'][0]):
        print(f"Resultado {i+1}:")
        print(f"Texto: {document}")
        print(f"Metadata: {results['metadatas'][0][i]}")
        print(f"ID: {results['ids'][0][i]}")
        print("-" * 50)

In [38]:
querys: list[str] = ["Cuanto cuesta comprar el juego?",
                     "Como se juega?",
                     "Cantidad de jugadores"]

for query in querys:
  print(f"Query: {query}")
  results = query_chromadb(collection, query, embedding_model, 1)

  display_results(results)

Query: Cuanto cuesta comprar el juego?
Resultado 1:
Texto: ¿Cuánto cuesta el juego en el mercado?
Metadata: {'answer': '59,99€.'}
ID: doc_19
--------------------------------------------------
Query: Como se juega?
Resultado 1:
Texto: ¿Qué tipo de caja tiene el juego?
Metadata: {'answer': 'Una caja rectangular de dos piezas (tapa y fondo).'}
ID: doc_20
--------------------------------------------------
Query: Cantidad de jugadores
Resultado 1:
Texto: ¿Cuál es el rango de jugadores permitido?
Metadata: {'answer': 'De 1 a 4 jugadores.'}
ID: doc_14
--------------------------------------------------


# 2. Preparación del RAG

## 2.1 Funciones generales

In [39]:
def get_key() -> str:
  dotenv.load_dotenv()
  key = os.getenv('AIK_HEADER') + os.getenv('AIK_CHUNK_1') + os.getenv('AIK_CHUNK_2') + os.getenv('AIK_CHUNK_3')

  return key

In [40]:
def augment_with_retrieved_data(chat_prompt: list, collection, query: str, embedding_model, top_k: int = 3):
    """
    Embebe información relevante de la base vectorial al prompt.
    """
    retrieval_results = query_chromadb(collection, query, embedding_model, top_k)

    relevant_data = " ".join(result for result in retrieval_results["documents"][0])

    chat_prompt.append({
        "role": "system",
        "content": f"Información adicional relevante: {relevant_data}"
    })

    return chat_prompt

In [41]:
mensajes = [
    {
        "role": "system",
        "content": "Sos un chatbot que se encarga de brindar informacion sobre el juego Las ruinas perdidas de Arnak, no dar respuestas largas",
    }
]

In [42]:
preguntas = [
    "¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?",
    "¿Cuáles son las mecánicas principales de Las Ruinas Perdidas de Arnak?",
    "¿Cómo se juega el modo de exploración en Las Ruinas Perdidas de Arnak?",
    "¿Cuántos jugadores pueden participar en Las Ruinas Perdidas de Arnak?",
    "¿Cuál es la duración aproximada de una partida?",
    "¿Qué componentes incluye el juego?",
    "¿Qué tan complejo es aprender a jugar Las Ruinas Perdidas de Arnak?",
    "¿Existen expansiones o contenido adicional para el juego?",
    "¿Qué estrategia se recomienda al comenzar una partida?",
    "¿Dónde se pueden comprar Las Ruinas Perdidas de Arnak?"
]

## 2.2 RAG basado en LLM

In [43]:
context = mensajes.copy()

In [44]:
def generate_text_with_openai(message):
  global context

  api_key = get_key()

  context = augment_with_retrieved_data(context, collection, message, embedding_model, 10)

  context.append({
      "role": "user",
      "content": message
  })

  client = OpenAI(api_key=api_key)

  response = client.chat.completions.create(model="gpt-3.5-turbo",
                                            messages=context)

  context.append(response.choices[0].message)

  print(f'User: {message}')
  print(f'Chat: {response.choices[0].message.content}')
  print('-'*50, '\n\n')

In [45]:
for pregunta in preguntas:
  generate_text_with_openai(pregunta)

User: ¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?
Chat: En Las Ruinas Perdidas de Arnak, los jugadores asumen el papel de exploradores y arqueólogos que se aventuran en una isla misteriosa en busca de ruinas, tesoros y artefactos. El objetivo principal del juego es descubrir y explorar los sitios arqueológicos, investigar las ruinas para obtener conocimiento, comprar artefactos valiosos, y por supuesto, obtener la mayor cantidad de puntos de victoria al final de la partida.
-------------------------------------------------- 


User: ¿Cuáles son las mecánicas principales de Las Ruinas Perdidas de Arnak?
Chat: Las mecánicas principales de Las Ruinas Perdidas de Arnak incluyen deckbuilding, exploración de losetas, gestión de recursos, obtención de puntos de victoria, y toma de decisiones estratégicas. Los jugadores deben planificar cuidadosamente sus acciones, gestionar sus recursos de forma eficiente y optimizar su mazo de cartas para lograr sus objetivos y obte

## 2.3 ChatBot con Árboles de Desición

In [46]:
data = data = pd.read_csv(os.path.join(os.getcwd(), 'data', 'preguntas_y_respuestas.csv'))

In [47]:
data.sample(5)

,Pregunta,Respuesta
68,What happens if a player cannot pay the cost o...,They cannot play the card.
64,How many starting cards does each player receive?,Each player receives 4 fear cards and 2 equipm...
48,¿Cómo se obtienen cartas de artefacto?,Pagando brújulas y activando su efecto inmedia...
66,What is the main mechanic this game combines?,Deck-building and worker placement.
16,¿Qué empresa publicó el juego originalmente?,Czech Games Edition (CGE).


In [48]:
vectorizer = TfidfVectorizer(max_features=5000)

X = vectorizer.fit_transform(data["Pregunta"])

In [49]:
data["Etiqueta"] = data["Respuesta"].astype("category").cat.codes  # Codifica categorías

y = data["Etiqueta"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [51]:
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [52]:
y_pred = classifier.predict(X_test)

unique_labels = sorted(y_test.unique())
target_names = data["Respuesta"].astype("category").cat.categories[unique_labels]

In [53]:
def predecir_respuesta(input_text: str) -> str:
    """
    Predice la respuesta para una pregunta usando el modelo Random Forest entrenado.
    """
    input_vector = vectorizer.transform([input_text])

    predicted_label = classifier.predict(input_vector)[0]

    respuesta = data["Respuesta"].astype("category").cat.categories[predicted_label]

    return respuesta

In [54]:
preguntas: list[str] = [
    "¿Cuántos tipos de tableros existen en el juego base?",
    "¿Qué tipo de objetos se encuentran en las cartas de artefactos?",
    "¿Cuántos ídolos pueden ser obtenidos durante el juego?",
    "¿Cuántas cartas de equipo se asignan a cada jugador al comenzar?",
    "¿Cuántos templos de diferentes colores están disponibles en el tablero?",
    "¿Qué mecánicas principales componen la jugabilidad de este juego?",
    "Cuantas losetas hay?"]

In [55]:
for pregunta in preguntas:
  print(f'Pregunta: {pregunta}')
  print(f'Respuesta: {predecir_respuesta(pregunta)}')
  print('-'*50, '\n\n')

Pregunta: ¿Cuántos tipos de tableros existen en el juego base?
Respuesta: Hay 12 ídolos en el tablero principal.
-------------------------------------------------- 


Pregunta: ¿Qué tipo de objetos se encuentran en las cartas de artefactos?
Respuesta: Botas, coches, barcos y aviones.
-------------------------------------------------- 


Pregunta: ¿Cuántos ídolos pueden ser obtenidos durante el juego?
Respuesta: Czech Games Edition (CGE).
-------------------------------------------------- 


Pregunta: ¿Cuántas cartas de equipo se asignan a cada jugador al comenzar?
Respuesta: Objetos antiguos que ayudan a los jugadores con habilidades únicas.
-------------------------------------------------- 


Pregunta: ¿Cuántos templos de diferentes colores están disponibles en el tablero?
Respuesta: 35 cartas.
-------------------------------------------------- 


Pregunta: ¿Qué mecánicas principales componen la jugabilidad de este juego?
Respuesta: Czech Games Edition (CGE).
------------------------

## 2.4 Chatbot en base a búsqueda semántica.

In [56]:
def busqueda_semantica(pregunta: str) -> str:
    pregunta_embedding = embedding_model.encode(pregunta)

    results = collection.query(
        query_embeddings=[pregunta_embedding],
        n_results=3
    )

    respuestas = [result["answer"] for result in results["metadatas"][0] if "answer" in result.keys()]
    respuestas_embeddings = [embedding_model.encode(respuesta) for respuesta in respuestas]

    similitudes = []

    for respuesta_embedding in respuestas_embeddings:
        sim = np.dot(pregunta_embedding, respuesta_embedding) / (np.linalg.norm(pregunta_embedding) * np.linalg.norm(respuesta_embedding))
        similitudes.append(sim)

    indices_ordenados = np.argsort(similitudes)[::-1]

    mejor_respuesta = respuestas[indices_ordenados[0]]

    return mejor_respuesta

In [57]:
preguntas: list[str] = ["Quien creó el juego las ruinas perdidas de arnak?",
                        "Cuales son los objetivo del juego",
                        "Cuantas cartas recibe cada jugador?"]

for pregunta in preguntas:
  print(f'Pregunta: {pregunta}')
  print(f'Respuesta: {busqueda_semantica(pregunta)}')
  print('-'*50, '\n\n')

Pregunta: Quien creó el juego las ruinas perdidas de arnak?
Respuesta: Un recurso para descubrir yacimientos y adquirir artefactos.
-------------------------------------------------- 


Pregunta: Cuales son los objetivo del juego
Respuesta: Obtener la mayor cantidad de puntos de victoria explorando y desbloqueando áreas del tablero.
-------------------------------------------------- 


Pregunta: Cuantas cartas recibe cada jugador?
Respuesta: Cada jugador recibe 4 cartas de miedo y 2 cartas de equipo.
-------------------------------------------------- 




## 2.5 ChatBot basado en agente

In [70]:
!curl -fsSL https://ollama.com/install.sh | sh

!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh

!chmod +x ollama_start.sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'


In [113]:
!nohup ./ollama_start.sh &

nohup: appending output to 'nohup.out'


In [116]:
!ollama pull llama3.2:latest > ollama.log

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏ 9.1 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  31 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  43 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  59 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕▏  88 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏ 102 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   6% ▕▏ 118 MB/

In [124]:
def wikipedia(query: str, lang: str = 'es') -> str:
    query = f"{query} para el juego las Ruinas perdidas de arnak"
    try:
        wikipedia.set_lang(lang)
        result = wikipedia.summary(query, sentences=3)
        return result
    except Exception as e:
        return "No se encontró información"

In [73]:
def get_csv_data(consulta: str) -> list[dict]:
    """
    Lee un archivo CSV y devuelve una lista de diccionarios, donde cada diccionario representa una fila.
    """
    pd_df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'preguntas_y_respuestas.csv'))

    return pd_df[pd_df['Pregunta'].str.contains(consulta, case=False)]

In [246]:
def get_vector(consulta: str) -> list[str]:
    docs = collection.query(
        query_texts=[str(consulta)],
        n_results=10
    )
    return ' '.join([x['answer'] for x in docs['metadatas'][0]])

In [247]:
llm = Ollama(
    model="llama3.2:latest",
    request_timeout=10.0,
    temperature=0.1,
    context_window=4096
)


Settings.llm = llm

!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &

In [248]:
tools = [
    FunctionTool.from_defaults(fn=get_vector, description="Information from database"),
]

In [249]:
agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt=
    """
    Estás diseñado para responder preguntas acerca del juego *Las Ruinas Perdidas de Arnak* utilizando **exclusivamente información de las bases de datos tabular, vectorial y de grafos brindada por las herramientas que se mencionan a continuación**. Asegúrate de seguir estrictamente las instrucciones para cada consulta.

     ## Herramientas disponibles:
    1. **get_vector**: Informacion de la base de datos vectorial. Parámetro (IMPORTANTE): recibe una string que representa el texto de consulta.

    ### Instrucciones para cada consulta:
    1. **Analiza la consulta** para determinar la información necesaria.
    2. **Usa las herramientas disponibles una por una**, pasando como parámetro **exactamente** la consulta recibida en formato de string.
    3. **Combina los resultados** en una respuesta final clara y concisa.

    ### Reglas adicionales:
    - **Nunca modifiques la consulta del usuario**: Cada vez que recibas una consulta nueva, **olvida la anterior**. La nueva consulta debe ser tratada de manera independiente y sin influencias previas.
    - **Prioriza las herramientas** según la relación temática con la consulta. Asegúrate de seleccionar la herramienta que mejor se ajuste a la consulta de acuerdo con las temáticas proporcionadas.
    - Si la pregunta requiere información de **más de una herramienta**, usa **todas** las herramientas necesarias para obtener una respuesta completa.
    - **Nunca** respondas con información que no provenga directamente de las herramientas.

    ### Formato de salida:
    La respuesta debe seguir estrictamente el siguiente ejemplo:

    #### Consulta 1:
    "¿Qué representan las losetas de ídolo?"
    - **Pensamiento (Thought)**: Necesito buscar información sobre los ídolos en las reglas generales del juego.
    - **Acción (Action)**: `get_vector`
    - **Entrada de acción (Action Input)**: "¿Qué representan las losetas de ídolo?"
    - **Observación (Observation)**: "Las losetas de ídolo representan beneficios inmediatos y otorgan puntos de victoria al final del juego."
    - **Respuesta final (Final Answer)**: Las losetas de ídolo representan beneficios inmediatos y otorgan puntos de victoria al final del juego.

    #### Consulta 2:
    "¿Quiénes diseñaron Las Ruinas Perdidas de Arnak?"
    - **Pensamiento (Thought)**: Necesito buscar información sobre los diseñadores en la base de datos de grafos.
    - **Acción (Action)**: `get_vector`
    - **Entrada de acción (Action Input)**: "¿Quiénes diseñaron Las Ruinas Perdidas de Arnak?"
    - **Observación (Observation)**: "Michal Štach y Michaela Štachová son los diseñadores del juego."
    - **Respuesta final (Final Answer)**: Michal Štach y Michaela Štachová son los diseñadores del juego.
    """,
              react_chat_history=False,
              context="""
              Sos un asistente que responde en español.
              """)


In [250]:
def predict(query):
    try:
        response = agent.chat(query)
        print(response)
    except Exception as e:
        print(f"Error al procesar la consulta: {str(e)}")

In [251]:
preguntas: list[str] = [
    "¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?",
    "¿Cuáles son las mecánicas principales de Las Ruinas Perdidas de Arnak?",
    "¿Cómo se juega el modo de exploración en Las Ruinas Perdidas de Arnak?",
    'How many player can play in Las Ruinas Perdidas de Arnak?',
    "¿What includes the game?",
]

In [252]:
for pregunta in preguntas:
  print(f'Pregunta: {pregunta}')
  predict(pregunta)
  print('-'*50, '\n\n')

Pregunta: ¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?
> Running step 320d5896-6fe5-46d0-867f-a3aa1100e277. Step input: ¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?
Thought: The current language of the user is: español. I need to use a tool to help me answer the question.
Action: get_vector
Action Input: {'consulta': AttributedDict([('title', 'Consulta sobre el juego Las Ruinas Perdidas de Arnak'), ('type', 'string')])}
Observation: Michal Štach y Michaela Štachová. 2 peones por jugador. Obtener la mayor cantidad de puntos de victoria explorando y desbloqueando áreas del tablero. 63,5×88 mm. Objetos antiguos que ayudan a los jugadores con habilidades únicas. Artefactos, equipo y cartas de miedo. 59,99€. Una caja rectangular de dos piezas (tapa y fondo). 35 cartas. Construcción de mazos y colocación de trabajadores.
> Running step 6d849cb1-cbc8-4e98-ad29-af4036c457fd. Step input: None
Thought: I can answer without using any more tools. I